In [5]:
import cudaq
from cudaq import spin
import numpy as np
from math import pi
from typing import List
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator, NullLocator
import matplotlib.patches as mpatches
import os
import pandas as pd
import pickle

In [6]:
for i in np.ndindex(2, 2, 2, 2):
    print(np.array(i))

ss = np.array([0, 1])
a = np.array(np.ndindex(2, 2, 2, 2))
# print(ss[a])

[0 0 0 0]
[0 0 0 1]
[0 0 1 0]
[0 0 1 1]
[0 1 0 0]
[0 1 0 1]
[0 1 1 0]
[0 1 1 1]
[1 0 0 0]
[1 0 0 1]
[1 0 1 0]
[1 0 1 1]
[1 1 0 0]
[1 1 0 1]
[1 1 1 0]
[1 1 1 1]


In [7]:
state = np.random.get_state()

np.random.uniform(0, 1, (2, 2))
print(np.random.randn(5))

np.random.set_state(state)
np.random.uniform(-2, 2, (2, 2))
print(np.random.randn(5))


[-1.93503718 -1.24301212  0.72376414  1.07972175 -0.29865399]
[-1.93503718 -1.24301212  0.72376414  1.07972175 -0.29865399]


In [8]:
a = 1.0
print(a.is_integer())

True


In [9]:
a = np.arange(25).reshape(5, 5)
b = np.array([1, 3, 1, 2, 1])
bb = np.diag(b)
print(a)
print(bb @ a @ bb)

print((b[None, :] * a) * b[:, None])

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
[[ 0  3  2  6  4]
 [15 54 21 48 27]
 [10 33 12 26 14]
 [30 96 34 72 38]
 [20 63 22 46 24]]
[[ 0  3  2  6  4]
 [15 54 21 48 27]
 [10 33 12 26 14]
 [30 96 34 72 38]
 [20 63 22 46 24]]


In [10]:
a = np.random.choice(50, 7, replace=False)
print(a)

[46 11 45  9 28  4 42]


In [11]:
df = pd.DataFrame({
    "A": [4, 1, 3, 2],
    "B": [40, 10, 30, 20]
})

df.sort_values(by="A", inplace=True)
print(df)

   A   B
1  1  10
3  2  20
2  3  30
0  4  40


In [8]:
df = pd.read_csv("./experiments_plateau_X/exp_0_L0.001_q1/report.csv")
print(df)
S = df[df["Assets"] == 7].shape[0]
N = df.loc[df["Assets"] == 7, "N"]
print(S, N)

Index(['Exp', 'Assets', 'Qubits', 'N', 'Sum_1', 'Sum_2', 'Coeff', 'Budget',
       'var', 'log_var'],
      dtype='object')
               Exp       N     Sum_1     Sum_2     Coeff       Budget  \
Assets Qubits                                                           
3.0    9.0     1.5  1500.0 -0.163121  2.884883  0.000060  1106.367040   
4.0    12.0    1.0  1500.0 -0.733525  0.483562  0.000085   999.717090   
5.0    15.0    1.0  1500.0  0.092308  0.121872  0.000059   997.727491   
6.0    18.0    1.0  1500.0  0.021536  0.053166  0.000049   993.493951   
7.0    21.0    1.0  1500.0  0.062054  0.032848  0.000031  1076.231997   

                    var   log_var  
Assets Qubits                      
3.0    9.0     0.001923 -2.716645  
4.0    12.0    0.000322 -3.493190  
5.0    15.0    0.000081 -4.173048  
6.0    18.0    0.000035 -4.464520  
7.0    21.0    0.000022 -4.867234  


In [14]:
a = spin.z(2)   

print(a.to_json())

[1.0, 1.0, 0.0, 1.0, 2.0, 1.0]


In [15]:
l, r, m = 0, 1000, 0

while m != r:
    m = (l+r)/2
    l = m
    print(m)

500.0
750.0
875.0
937.5
968.75
984.375
992.1875
996.09375
998.046875
999.0234375
999.51171875
999.755859375
999.8779296875
999.93896484375
999.969482421875
999.9847412109375
999.9923706054688
999.9961853027344
999.9980926513672
999.9990463256836
999.9995231628418
999.9997615814209
999.9998807907104
999.9999403953552
999.9999701976776
999.9999850988388
999.9999925494194
999.9999962747097
999.9999981373549
999.9999990686774
999.9999995343387
999.9999997671694
999.9999998835847
999.9999999417923
999.9999999708962
999.9999999854481
999.999999992724
999.999999996362
999.999999998181
999.9999999990905
999.9999999995453
999.9999999997726
999.9999999998863
999.9999999999432
999.9999999999716
999.9999999999858
999.999999999993
999.9999999999965
999.9999999999982
999.9999999999991
999.9999999999995
999.9999999999998
999.9999999999999
1000.0


In [99]:
P = np.array([148.2365, 178.1284, 236.5621])
MI, MA = 125, 250*7
target = 9

def rdd(a, coeff, order = 7, s = 1e-9):
    return round(a + coeff * s, order)

mi, ma = MI, MA
cou = 0
mid = (mi + ma)/2
while (N := np.sum(np.int32(np.floor(np.log2(mid/P))) + 1)) != target:
    if N < target:
        mi = mid
    else:
        ma = mid
    # print()
    mid = rdd((mi + ma)/2, 0, 7)
    cou += 1
MID = mid
print(MID, cou)

1140.625 2


In [85]:
print(round(3.75 + 1e-9, 1))
print(np.round(3.35 + 1e-9, 1))

3.8
3.4


In [92]:
mi, ma = MI, MID
cou = 0
mid = (mi + ma)/2
while mid != ma:
    if np.sum(np.int32(np.floor(np.log2(mid/P))) + 1) < target:
        mi = mid
    else:
        ma = mid
    mid = rdd((mi + ma)/2, 1, 7)
    cou += 1
MIN = mid
print(MIN, cou)


946.2484 34


In [98]:
mi, ma = MID, MA
cou = 0
mid = (mi + ma)/2
while mid != ma:
    if np.sum(np.int32(np.floor(np.log2(mid/P))) + 1) > target:
        ma = mid
    else:
        mi = mid
    mid = rdd((mi + ma)/2, 1, 7)
    cou += 1
    print(mi, ma, mid)
MAX = mid
print(MAX, cou)

1140.625 1445.3125 1292.96875
1140.625 1292.96875 1216.796875
1140.625 1216.796875 1178.7109375
1178.7109375 1216.796875 1197.7539063
1178.7109375 1197.7539063 1188.2324219
1178.7109375 1188.2324219 1183.4716797
1183.4716797 1188.2324219 1185.8520508
1185.8520508 1188.2324219 1187.0422364
1185.8520508 1187.0422364 1186.4471436
1185.8520508 1186.4471436 1186.1495972
1185.8520508 1186.1495972 1186.000824
1185.8520508 1186.000824 1185.9264374
1185.8520508 1185.9264374 1185.8892441
1185.8892441 1185.9264374 1185.9078408
1185.8892441 1185.9078408 1185.8985425
1185.8892441 1185.8985425 1185.8938933
1185.8892441 1185.8938933 1185.8915687
1185.8915687 1185.8938933 1185.892731
1185.8915687 1185.892731 1185.8921499
1185.8915687 1185.8921499 1185.8918593
1185.8918593 1185.8921499 1185.8920046
1185.8918593 1185.8920046 1185.891932
1185.891932 1185.8920046 1185.8919683
1185.8919683 1185.8920046 1185.8919865
1185.8919865 1185.8920046 1185.8919956
1185.8919956 1185.8920046 1185.8920001
1185.8919956 1

In [33]:
print(np.sum(np.int32(np.floor(np.log2(946.2484/P))) + 1))

9
